# correlation of psd slope with age

In [1]:
%matplotlib inline
import os
import glob
import numpy as np
import scipy as sp
import seaborn as sb
import matplotlib.pyplot as plt
from scipy import signal
from scipy import io
from sklearn import linear_model, datasets

#### Data Importing

In [2]:
subj = {} # Dictionary that holds all subjects
matfiles = []
for root, dirs, files in os.walk('../data/extClfiltCAR-mat/'):
    matfiles += glob.glob(os.path.join(root, '*.mat'))

for i in range(len(matfiles)):
    datafile = sp.io.loadmat(matfiles[i])
    
    subj[i] = {}
    subj[i]['name']  = np.squeeze(datafile['name'])
    subj[i]['srate'] = np.squeeze(datafile['srate'])
    subj[i]['data']  = np.squeeze(datafile['data'])    

#### PSD Slope Results

In [3]:
def compute_slope(f, psd):
    model_ransac = linear_model.RANSACRegressor(linear_model.LinearRegression())
    model_ransac.fit(f[:50*2], np.log10(psd[:50*2]))
    fit_line = model_ransac.predict(f)
    return model_ransac.estimator_.coef_[0][0]

In [4]:
srate = 512
for i in subj:
    for ch in range(len(subj[i]['data'])):
        # For all subjects, compute the slope of each channel
        subj[i][ch] = {}
        
        f, psd = sp.signal.welch(subj[i]['data'][ch], srate, nperseg=srate*2, noverlap=srate, window='hamming')
        f   = f.reshape((len(f), 1))
        psd = psd.reshape((len(psd), 1))
        
        subj[i][ch]['f'] = f
        subj[i][ch]['psd'] = psd
        subj[i][ch]['slope'] = compute_slope(f, psd)
        
    # Now compute each subject's mean slope    
    slopes = []
    for ch in range(len(subj[i]['data'])):
        slopes.append(subj[i][ch]['slope'])
    subj[i]['slope'] = np.mean(slopes)